In [1]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

from constant_prompts import make_constant_prompts
from util.generate import generate_fast # adding

import json
import pandas as pd
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [2]:
MODEL_NAME = "gpt2-xl" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [4]:
with open('data/counterfact-selected-qual.json') as json_file:
   cf_data = json.load(json_file)

reldf = pd.read_csv("counterfact/counterfact-selected-relations.csv")
print(len(cf_data))

172


In [26]:
from evaluate import load
bertscore = load("bertscore")

# def generate_text(texts, model, tok):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens=15,
#                                        num_return_sequences = 5,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)

tokenizer.pad_token = tokenizer.eos_token

def gen_constant_prompts(subject, subject_type, orig_target, model, tokenizer, top_n = 10):

    prompts = make_constant_prompts(subject, subject_type)
    
    generations = generate_fast(model, tokenizer, prompts, n_gen_per_prompt = 5, max_out_len = 25)
    gens_per = len(generations) // len(prompts)
    out = []

    for i in range(len(prompts)):
        gens = generations[i*gens_per:i*gens_per+gens_per]
        prompt = prompts[i]
        # max_probs = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
        # mean_probs = torch.mean(max_probs.values)
        predictions = [g[len(prompts[i]):] for g in gens]
        references = [orig_target]*len(gens)
        results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased") # "distilbert-base-uncased"
        val = max(results["recall"])

        resdict = dict()
        resdict["prompt"] = prompts[i]
        resdict["gens"] = gens
        resdict["recall"] = results["recall"]
        resdict["val"] = val

        out.append(resdict)

    out_sorted = sorted(out, key=lambda d: d['val'])

    return(out_sorted[:top_n])


def pick_min_similarity_text(gens):
    min_val = min(gens["recall"])
    idx = gens["recall"].index(min_val)    
    gens["gens"] = [gens["gens"][idx]]
    gens["val"] = min_val
    out = {k: v for k, v in gens.items() if k != 'recall'}
    
    return(out)


    
# gen_constant_prompts("Lebron James", "person", "basketball", model, tokenizer, 6)

In [45]:
for i in range(0,5): print(i)

0
1
2
3
4


In [27]:
# write a new function to compute max probability of token in certain generations
def encode_token(token:str, tokenizer):
    
    if token[0] != " ": # pad token
        token = " " + token
        
    token_id = tokenizer(token)["input_ids"]
    return(token_id)
    
def token_logits(texts, token, tokenizer, start_ind = 0):
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        model_out = model(encoding["input_ids"])
        logits = model_out.logits
        logprobs = F.log_softmax(logits, -1)

    token_id = encode_token(token, tokenizer)
    
    return(logprobs[:, start_ind:, token_id])


def token_max_prob(texts, token, tokenizer, start_ind = 0):
    token_id = encode_token(token, tokenizer)
    logits = token_logits(texts, token, tokenizer)
    # logits = logits[:, start_ind:, token_id]
    out = torch.max(logits, 1)
    
    return(out)

def count_tokens(text, tokenizer):
    if type(text)==list:
        assert len(text)==1, "count_tokens() only meant to count tokens of one string at a time"
        
    encoding = tokenizer(text, return_tensors='pt')
    return(len(encoding["input_ids"][0]))


def choose_lowest_max_prompt(gens_dict, orig_target, tokenizer):
    gens = gens_dict["gens"]
    prompt = gens_dict["prompt"]
    logits = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
    
    gidx = torch.min(logits.values, 0).indices.squeeze()
    # tidx = logits.indices[gidx].squeeze()
    
    encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
    candidate_prompts = tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt, tokenizer) + logits.indices[i]] for i in range(len(gens))])
    
    return(candidate_prompts[gidx])


def choose_highest_max_prompt(gens_dict, orig_target, tokenizer):
    gens = gens_dict["gens"]
    prompt = gens_dict["prompt"]
    logits = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
    
    gidx = torch.max(logits.values, 0).indices.squeeze()
    # tidx = logits.indices[gidx].squeeze()
    
    encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
    candidate_prompts = tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt, tokenizer) + logits.indices[i]] for i in range(len(gens))])
    
    return(candidate_prompts[gidx])


In [30]:
subject = "Lebron James"
orig_target = "basketball"
testgens = gen_constant_prompts(subject, "person", orig_target, model, tokenizer, 6)

In [320]:
[choose_lowest_max_prompt(g, orig_target, tokenizer) for g in testgens]

['Lebron James starred in the first episode of "The',
 "Lebron James's mother was a drug addict,",
 "Lebron James invests in his own shoe company, Roc Nation Sports LeBron James' shoe company, Roc Nation, is now worth more than $1 billion, the New York Times reported. The Times reported that James",
 'Lebron James wrote: "I don\'t care what they say. I\'m going to the White House. I will not go to the White House. I have a great relationship with all',
 "Lebron James's body is still on the court after the game against the",
 'Lebron James earned their second straight title, and the first']

In [327]:
pick_min_similarity_text(testgens[5])

{'prompt': 'Lebron James earned their',
 'gens': ['Lebron James earned their third straight trip to the Finals with a Game 6 victory. LeBron James, Kevin Love, and Kyrie Irving scored 30 points each to lead the Cavaliers past the Warriors, 97-95, in a best-of-'],
 'val': 0.6208294630050659}

In [31]:
testgen = testgens[4]
gens = testgen["gens"]
prompt = testgen["prompt"]

x = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))

print(choose_lowest_max_prompt(testgen, orig_target, tokenizer))
torch.exp(torch.min(x.values))

Lebron James's best friend is the same person who's


tensor(0.0152, device='cuda:0', dtype=torch.float16)

In [256]:
print(choose_highest_max_prompt(testgen, orig_target, tokenizer))
torch.exp(torch.max(x.values))

Charles Darwin's race and ethnicity is a central issue in the ongoing debate about his theory


tensor(0.0190, device='cuda:0', dtype=torch.float16)

In [25]:
def log1mexp(x):
    """Numerically accurate evaluation of log(1 - exp(x)) for x < 0.
    See [Maechler2012accurate]_ for details.
    """
    mask = x < 0
    return torch.where(
        mask,
        (-x.expm1()).log(),
        (-x.exp()).log1p(),
    )

## what about the probability of it not occuring in the whole string?
# l = token_logits(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
# tt = torch.tensor([-.25])
# print(torch.log(1 - torch.exp(tt)))
# print(log1mexp(tt))
# print(torch.log(-torch.expm1(tt)))
# prob_not_appearing = torch.exp(torch.sum(log1mexp(l), 1))
# torch.max(prob_not_appearing,0).indices.item()

In [265]:
gens[torch.max(prob_not_appearing,0).indices.item()]

'Charles Darwin\'s race and ethnicity is a matter of debate. In his book The Descent of Man, he wrote that "The most probable view, I am inclined to think, is that the human species has undergone no great change since the appearance of'

In [186]:
encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt,tokenizer) + x.indices[i]] for i in range(len(gens))])

['iCloud is aimed at the cloud computing market, where the cost of computing',
 "iCloud is aimed at helping users store and sync photos from multiple devices, and it's a good",
 'iCloud is aimed at providing a secure and reliable storage for all of your files, including music',
 'iCloud is aimed at helping you get the most out of your data, and to',
 'iCloud is aimed at making it easier to access']

In [59]:
def logprob_target_not_appear(texts, target, prompt, model, tokenizer):
    ## what about the probability of it not occuring in the whole string?
    l = token_logits(texts, target, tokenizer, count_tokens(prompt, tokenizer))
    return(torch.sum(log1mexp(l), 1))

# def get_least_likely(texts, target, model, tokenizer):
    

least_likely = gens[torch.max(logprob_target_not_appear(gens, orig_target, prompt, model, tokenizer),0).indices.item()]
print(least_likely)
logprob_target_not_appear(least_likely, orig_target, prompt, model, tokenizer).item()

Lebron James's best friend is LeBron James. And he was in Cleveland for the Cavs' game with the Raptors on


-0.00390625

In [10]:
tokenizer.pad_token = tokenizer.eos_token

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(25563)

for x in tqdm(cf_data):

    rel_id = x["requested_rewrite"]["relation_id"]
    subject = x["requested_rewrite"]["subject"]
    orig_target = x["requested_rewrite"]["target_true"]["str"]
    subject_type = reldf.loc[lambda x: x.relation_id == rel_id].subj_type.item()


    cprompts = gen_constant_prompts(subject, subject_type, orig_target, model, tokenizer, 6)
    
    x["subj_const_prompts"] = {MODEL_NAME: cprompts}
    x["low_target_prompts"] = {MODEL_NAME: [choose_lowest_max_prompt(g, orig_target, tokenizer) for g in cprompts]}



100%|██████████| 172/172 [04:41<00:00,  1.63s/it]


In [11]:
with open("data/counterfact-selected-qual.json", "w") as file:
    json.dump(cf_data, file, indent=4)

In [27]:
## test stuff

import sentence_transformers as st

def sentence_similarity_matrix(sentences1, sentences2):
    from sentence_transformers import SentenceTransformer, util
    smodel = SentenceTransformer('all-MiniLM-L6-v2')

    #Compute embedding for both lists
    embeddings1 = smodel.encode(sentences1, convert_to_tensor=True)
    embeddings2 = smodel.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    return(cosine_scores)


def avg_sentence_similarity(sentences1, sentences2):
    return(torch.mean(sentence_similarity_matrix(sentences1, sentences2)))


# def generate_sc_text(texts, model, tok, max_new_tokens=15, num_return_sequences = 5):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens = max_new_tokens,
#                                        num_return_sequences = num_return_sequences,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)


def calc_subj_gen_similarity(model, tok, gen_prompts, orig_gens):
    sims = []
    for i in range(len(gen_prompts)):
        gens = generate_fast(model, tok, [gen_prompts[i]], n_gen_per_prompt = 5, max_out_len = 25)
        gens = [g[len(gen_prompts[i]):] for g in gens] # just use the generated part, not the original prompt
        sentence_similarity = avg_sentence_similarity(gens, orig_gens[i])
        sims.append(sentence_similarity.item())

    mean_sim = sum(sims)/len(sims)
    
    return(gens, mean_sim)

In [28]:
tokenizer.pad_token = tokenizer.eos_token
record = cf_data[0]
prompts = [record["subj_const_prompts"][MODEL_NAME][i]["prompt"] for i in range(0,6)]
orig_gens = [record["subj_const_prompts"][MODEL_NAME][i]["gens"] for i in range(0,6)]


# generate_fast(model, tokenizer, [prompts[0]])
calc_subj_gen_similarity(model, tokenizer, prompts, orig_gens)

(['    $1,000,000 worth of cocaine from the       ',
  ' Category:Living people\nCategory:1954 births\nCategory:British male sailors (sport',
  '\nhis first guitar when he was 14 and played with the school band. He went on to play in',
  '\nhis first house for $25,000 in the late 1970s. He lived in it for three',
  ' \na new BMW M3 George Michael bought a new BMW M3 The M'],
 0.31336965163548786)

In [21]:
orig_gens

[[" St. George's School, Liverpool, and the University of Sheffield, where",
  " King's School, Bruton, Somerset, and at the Royal College of",
  " the independent St George's School, Weybridge, Surrey. He then",
  " St. John's School, Leatherhead and the Royal College of Art,",
  ' the Cambridge High School for Boys in New Addington, London. At Cambridge'],
 [' a high-school teacher, his father a high-school principal, and',
  ' the singer Grace Jones; his father was Afro-Guyanese.',
  ' a devout Christian and taught her son her beliefs. She raised her son in',
  " a successful interior decorator. George Michael's father was a successful architect.",
  ' born in London, England. He was born on May 27, 1958.'],
 [' a music composer and his mother, Cynthia, was a successful model and beauty',
  ' a well-known and much-respected jazz musician, composer and arranger',
  ' a doctor.\n\nGeorge Michael was born in north London on September 21',
  ' the musician George Michael.\n\nI remember 

In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [66]:
def calc_bertscore_recall(model, tok, gen_prompts, ref):
    sims = []
    for i in range(len(gen_prompts)):
        gens = generate_fast(model, tok, [gen_prompts[i]], n_gen_per_prompt = 5, max_out_len = 25) # not great to have this hardcoded

        references = [ref]*len(gens)
        results = bertscore.compute(predictions=gens, references=references, model_type="distilbert-base-uncased") # "distilbert-base-uncased"

        sims.extend(results["recall"])

    mean_sim = sum(sims)/len(sims) # compute average recall for all prompts
    
    return(mean_sim)

calc_bertscore_recall(model, tokenizer, [prompt]*2, "baseball")

0.5992435336112976